# Analyze Trader Activity

Extract and analyze trader activity from explorer_blocks.

In [1]:
import json
import msgpack
import lz4.frame
from pathlib import Path
from collections import Counter, defaultdict
from decimal import Decimal
import pandas as pd

SAMPLES_DIR = Path('../hyperliquid_samples/explorer_blocks')

In [2]:
def load_blocks(filepath):
    filepath = Path(filepath)
    if filepath.suffix == '.json':
        with open(filepath) as f:
            return json.load(f)
    elif filepath.suffix == '.lz4':
        with open(filepath, 'rb') as f:
            data = lz4.frame.decompress(f.read())
            return msgpack.unpackb(data, raw=False)
    elif filepath.suffix == '.rmp':
        with open(filepath, 'rb') as f:
            return msgpack.unpack(f, raw=False)

# Load recent blocks
blocks = load_blocks(SAMPLES_DIR / '811681900.json')
print(f"Loaded {len(blocks)} blocks")

Loaded 100 blocks


## Extract Trader Activity

In [3]:
def extract_trader_activity(blocks):
    """
    Extract activity per trader from blocks.
    Returns dict: address -> {orders, cancels, volume_usd, assets_traded}
    """
    activity = defaultdict(lambda: {
        'orders': 0,
        'cancels': 0,
        'modifies': 0,
        'assets': set(),
        'buy_orders': 0,
        'sell_orders': 0
    })
    
    for block in blocks:
        for tx in block.get('txs', []):
            user = tx.get('user')
            if not user:
                continue
                
            for action in tx.get('actions', []):
                action_type = action.get('type')
                
                if action_type == 'order':
                    for order in action.get('orders', []):
                        activity[user]['orders'] += 1
                        activity[user]['assets'].add(order.get('a'))
                        if order.get('b'):
                            activity[user]['buy_orders'] += 1
                        else:
                            activity[user]['sell_orders'] += 1
                            
                elif action_type in ('cancel', 'cancelByCloid'):
                    activity[user]['cancels'] += len(action.get('cancels', []))
                    
                elif action_type in ('modify', 'batchModify'):
                    activity[user]['modifies'] += 1
    
    # Convert sets to counts
    for user in activity:
        activity[user]['num_assets'] = len(activity[user]['assets'])
        activity[user]['assets'] = list(activity[user]['assets'])
    
    return dict(activity)

activity = extract_trader_activity(blocks)
print(f"Found {len(activity)} unique traders")

Found 679 unique traders


In [4]:
# Convert to DataFrame for analysis
df = pd.DataFrame.from_dict(activity, orient='index')
df.index.name = 'address'
df = df.reset_index()
df = df.sort_values('orders', ascending=False)
df.head(20)

,address,orders,cancels,modifies,assets,buy_orders,sell_orders,num_assets
12,0x362c5006ed0a405c35df4a138cb38d891f1a1a20,5186,240,0,[0],5186,0,1
2,0x13558be785661958932ceac35ba20de187275a42,5141,317,0,"[0, 25, 214, 159]",5131,10,4
14,0x1a2e6afa298b1cc50939b4b2b0b430e3c1ef3459,4959,232,0,[0],4959,0,1
11,0x7dd6f02fef939cea86f0243358255f6b57a6cf0d,4888,158,0,[0],4888,0,1
60,0xaf2ac71f62f341e5823d6985492409e92c940447,4734,183,0,[0],4734,0,1
24,0xd47c1e51b759cb3754bb045ae57bc05aadcf58c2,4641,148,0,[0],4641,0,1
10,0xe29d84a6f37858d7f8017be454ee35726511e525,3000,255,0,[0],3000,0,1
16,0x4d969e59b312970cc07af266938a4433ccae0b4c,2982,71,0,[0],2982,0,1
40,0xa289ee1e56c0d5d041db762e6123e78af0f7d9ad,2953,546,0,[0],2944,9,1
68,0x54e3f54df4b10ff63f7284733ed735e8e1507c20,2772,399,0,[0],2772,0,1


In [5]:
# Top traders by order count
print("Top 10 Traders by Order Count:")
for _, row in df.head(10).iterrows():
    print(f"  {row['address'][:10]}... : {row['orders']:,} orders, {row['num_assets']} assets")

Top 10 Traders by Order Count:
  0x362c5006... : 5,186 orders, 1 assets
  0x13558be7... : 5,141 orders, 4 assets
  0x1a2e6afa... : 4,959 orders, 1 assets
  0x7dd6f02f... : 4,888 orders, 1 assets
  0xaf2ac71f... : 4,734 orders, 1 assets
  0xd47c1e51... : 4,641 orders, 1 assets
  0xe29d84a6... : 3,000 orders, 1 assets
  0x4d969e59... : 2,982 orders, 1 assets
  0xa289ee1e... : 2,953 orders, 1 assets
  0x54e3f54d... : 2,772 orders, 1 assets


In [6]:
# Distribution of order counts
print("\nOrder Count Distribution:")
print(df['orders'].describe())


Order Count Distribution:
count     679.000000
mean      136.343152
std       561.418464
min         0.000000
25%         1.000000
50%         3.000000
75%        17.000000
max      5186.000000
Name: orders, dtype: float64


## Order Analysis

In [7]:
def extract_orders(blocks):
    """Extract all orders with details"""
    orders = []
    
    for block in blocks:
        block_time = block['header']['block_time']
        block_height = block['header']['height']
        
        for tx in block.get('txs', []):
            user = tx.get('user')
            
            for action in tx.get('actions', []):
                if action.get('type') == 'order':
                    for order in action.get('orders', []):
                        orders.append({
                            'block_time': block_time,
                            'block_height': block_height,
                            'user': user,
                            'asset': order.get('a'),
                            'side': 'buy' if order.get('b') else 'sell',
                            'price': order.get('p'),
                            'size': order.get('s'),
                            'reduce_only': order.get('r'),
                            'tif': order.get('t', {}).get('limit', {}).get('tif'),
                            'cloid': order.get('c')
                        })
    
    return orders

orders = extract_orders(blocks)
print(f"Extracted {len(orders):,} orders")

Extracted 92,577 orders


In [8]:
orders_df = pd.DataFrame(orders)
orders_df.head()

,block_time,block_height,user,asset,side,price,size,reduce_only,tif,cloid
0,2025-11-28T22:44:28.368940020,811681801,0x847bd1ef4fede0fe191129894d63044843f4d473,1,buy,3034.7,2.5966,False,Alo,0x000000000000000000000000042ef3a0
1,2025-11-28T22:44:28.368940020,811681801,0x07fd7e702ba749ffa49c3a6c17fcd9e6c7b7082a,1,buy,3033.5,20.5319,False,Alo,0x00000000000000000143674b893ffac4
2,2025-11-28T22:44:28.368940020,811681801,0x13558be785661958932ceac35ba20de187275a42,0,buy,90941,0.01185,False,Alo,0x00000000239844b500001d671a7e1c90
3,2025-11-28T22:44:28.368940020,811681801,0x13558be785661958932ceac35ba20de187275a42,0,buy,90940,0.01185,False,Alo,0x00000000239844b600001d671a7e1c90
4,2025-11-28T22:44:28.368940020,811681801,0x13558be785661958932ceac35ba20de187275a42,0,buy,90939,0.01185,False,Alo,0x00000000239844b700001d671a7e1c90


In [9]:
# Order types distribution
print("Order Types (TIF):")
print(orders_df['tif'].value_counts())

Order Types (TIF):
tif
Alo               90667
Ioc                1622
Gtc                 267
FrontendMarket       18
Name: count, dtype: int64


In [10]:
# Buy vs Sell
print("\nBuy vs Sell:")
print(orders_df['side'].value_counts())


Buy vs Sell:
side
buy     84507
sell     8070
Name: count, dtype: int64


In [11]:
# Most traded assets
print("\nTop 10 Assets by Order Count:")
print(orders_df['asset'].value_counts().head(10))


Top 10 Assets by Order Count:
asset
0        66910
1         7508
5         6640
25        1799
159        608
10107      532
60         370
214        332
174        329
10156      277
Name: count, dtype: int64


## Identify Potential Fills (IOC Orders)

IOC (Immediate or Cancel) orders that cross the spread result in fills.
ALO (Add Liquidity Only) orders never fill immediately.

In [12]:
# IOC orders are potential takers
ioc_orders = orders_df[orders_df['tif'] == 'Ioc']
print(f"IOC orders (potential fills): {len(ioc_orders):,}")

# ALO orders are makers only
alo_orders = orders_df[orders_df['tif'] == 'Alo']
print(f"ALO orders (makers only): {len(alo_orders):,}")

IOC orders (potential fills): 1,622
ALO orders (makers only): 90,667


In [13]:
# IOC order details
print("\nSample IOC Orders:")
ioc_orders.head(10)


Sample IOC Orders:


,block_time,block_height,user,asset,side,price,size,reduce_only,tif,cloid
922,2025-11-28T22:44:28.368940020,811681801,0x223537ac9a856c31f4043e86ced86bb29f06653e,122,buy,0.28107,4851,False,Ioc,0x0000000000000000000000000444a764
923,2025-11-28T22:44:28.368940020,811681801,0x14ecb8b25e270b31cc270e3dd3ee220dd5dce16e,0,buy,90934,0.782,False,Ioc,0x0000000000004bb30000147e62f51c92
924,2025-11-28T22:44:28.368940020,811681801,0xa775d1bd91ee2cc4cae6113e5fd9c0c17b355277,5,buy,137.51,35.33,False,Ioc,0x000000000000000000000000042c5415
925,2025-11-28T22:44:28.368940020,811681801,0xa775d1bd91ee2cc4cae6113e5fd9c0c17b355277,0,buy,90931,0.02414,False,Ioc,0x000000000000000000000000042c5412
926,2025-11-28T22:44:28.368940020,811681801,0xb6a766f531fa8e222f460df11d62b0f84b7b65f3,1,buy,3034.7,12.82,False,Ioc,0x00000000000f6d810000134dcb731c93
927,2025-11-28T22:44:28.368940020,811681801,0xa775d1bd91ee2cc4cae6113e5fd9c0c17b355277,0,buy,90933,0.02414,False,Ioc,0x000000000000000000000000042c5413
928,2025-11-28T22:44:28.368940020,811681801,0x5306cb5adfbaf940ce3dba0ac41d903dc70a766e,0,buy,90929,0.00021,False,Ioc,0x559a63adef6784e7615f6659672fe700
929,2025-11-28T22:44:28.368940020,811681801,0xb6a766f531fa8e222f460df11d62b0f84b7b65f3,1,buy,3034.7,12.82,False,Ioc,0x00000000000f6d830000134dcb731c93
930,2025-11-28T22:44:28.368940020,811681801,0xa775d1bd91ee2cc4cae6113e5fd9c0c17b355277,25,buy,2.1816,1521,False,Ioc,0x000000000000000000000000042c541c
931,2025-11-28T22:44:28.368940020,811681801,0xa775d1bd91ee2cc4cae6113e5fd9c0c17b355277,25,buy,2.1814,1521,False,Ioc,0x000000000000000000000000042c540b
